<a href="https://colab.research.google.com/github/COGS118A/Group004-Sp22/blob/Dhruva's-branch-Testing/Final_Project_group004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COGS 118A- Project Checkpoint

# Names

- Daniel Milton
- Isabella Gonzalez
- Dhruva Kolikineni
- Harini Adivikolanu
- Brandon Rocchio

# Abstract 
Classification involves predicting discrete class labels for unlabeled data. given information on the data. The data we are working with is information from individuals' shopping trips at Walmart. The data is broken down into 7 observations, one of which is the trip type which tells us what type of shopping trip this customer was on, visit number which organizes the data into individual shopping trips, weekday that the trip was done on, UPC number of the item purchased, department of purchase and the fineline number which is a number that Walmart made helping us specify the items purchased. Given certain data such as what weekday it is, what department it was in, and what the UPC was, our goal is to be able to predict what type of shopping trip someone was on based off of a couple pieces of data.



# Background

Although supervised classification methods have been studied greatly throughout the years, research specific to this problem, on classifying a grocery trip based on the items bought remains low. There are multiple classification algorithms we can attempt in order to classify the 38 different types of shopping trips there are. K-nearest neighbors, decision trees, random forest classifiers, neural networks and logistic regression are commonly used in order to solve classification problems. Due to the low amount of research done in this specific problem area, this background section will be a small literature review of potential classification algorithms to use for our problem.

Firstly, K Nearest Neighbors works based on the idea that for a target variable, the k number of patterns nearest to that target variable can provide useful information in order to properly classify the target variable. KNN assigns the target variable the classification of the majority of the nearest neighbors<a name="first"></a>[<sup>[1]</sup>](#firstnote). The downfalls of KNN are that there is no right 'K' to choose and it can be computationally inefficient. Second, Decision Trees are another classification method we want to attempt, these are popular due to their good accuracy scores and their computational efficiency<a name="second"></a>[<sup>[2]</sup>](#secondnote). The random forest classifier works by using multiple tree classifiers where each classifier is generated by using a random vector and each tree vottes for the most 'popular' class to classify an input vector<a name="third"></a>[<sup>[3]</sup>](#thirdnote). This paper uses random forests to classify remote sensing which they concluded was just as accurate as using a support vector machine.

Neural Networks and logistic regression are other potential algorithms we would like to try to classif our data with, there was no literature on any similar classification task to our project but a paper revealed that these two algorithms share common roots in statistical pattern recognition and that neural networks can be seen as a type of generalization from logistic regression<a name="fourth"></a>[<sup>[4]</sup>](#fourthnote). Lastly, upon multiple attempts to find related work, Cui et al attempted this trip type classification using deep embedding logistic regression which incorporates logistic regression into a deep and narrow neural network<a name="fifth"></a>[<sup>[5]</sup>](#fifthnote). We are hoping as we implement some of these algorithms to produce results and a discussion that can help future research for stores like Walmart to improve customers' shopping experiences or help understand/solve problems similar to this.

# Problem Statement

Walmart currently employs a proprietary method to catogorize shopping trips into 38 distinct types. We have set out to create a clustering/catagorization model that, given a limited set of customer behavior features, predicts the shopping trip types.

As an example for what these trip types may be: a customer may make a small daily dinner trip, a weekly large grocery trip, a trip to buy gifts for an upcoming holiday, or a seasonal trip to buy clothes.

#Setup

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn import *
import io
import tensorflow as tp
from tensorflow import keras

# Data


## Data Preprocessing:

In [ ]:
## Load in train dataset
train = pd.read_csv((r'/content/train.csv'))

## Drop Upc column
#train.drop(columns=['Upc'], inplace = True)


## Find all rows with null values
null_vals = train[train.isna().any(axis=1)]

## I found from the above code that there is 4129 rows
## with null values. From this I found that Department Descriptions of
## PHARMACY RX always has a null FinelineNumber


## A check of all unique values in these columns to make sure if any
## further cleaning of the strings is needed
department_unique = train['DepartmentDescription'].unique()
weekday_unique = train['Weekday'].unique()
scancount_unique = train['ScanCount'].unique()

## Function to change the week days to quantitative variables, which allows us
## to do analysis on the Weekday column
def weekday_int_converter(x):
  if x=='Monday':
    return 0
  elif x=='Tuesday':
    return 1
  elif x=='Wednesday':
    return 2
  elif x=='Thursday':
    return 3
  elif x=='Friday':
    return 4
  elif x=='Saturday':
    return 5
  elif x=='Sunday':
    return 6 


train['Weekday'] = train['Weekday'].apply(weekday_int_converter)

## Filtered the train csv of all null values in the Department 
## Description column
train = train[train['DepartmentDescription'].notna()]


## We found there to be 60367 duplicate rows.
duplicates = train[train.duplicated()]

## We decided not to drop the duplicates as they may provide valuable data.
#train_duplicates_dropped = train.drop_duplicates()


## Add a column indicated if the trip includes return
train['Return'] = train['ScanCount']
train
def convert(x):
    if x >= 0:
        return 0
    else:
        return 1
train['Return'] = train['ScanCount'].apply(convert)
train
## Test any variable to see results
train.shape
print(train.head())
print(train["DepartmentDescription"].unique())


FileNotFoundError: ignored

In order to clean our data, we imported the training data set, dropped the UPC column since we determined it was not a significant variable, changed weekdays into numerical categories, dropped all NA values as well as duplicate rows. 

##Feature Engineering

In [ ]:
#temp = pd.get_dummies(train, columns=['DepartmentDescription'])
#temp  = temp.dropna()
#temp.isnull().values.any()
#temp

In [ ]:
## Feature Engineering




## Create a new df to append a groupby series of the sum of items bought in each trip
items_bought_each_trip = pd.DataFrame()



## Groupby function asssigned to new col
items_bought_each_trip['item_sum'] = train.groupby(['VisitNumber'])['ScanCount'].sum()


## This merge creates a df that makes only one row for each trip containing its sum of items
merged = items_bought_each_trip.reset_index().merge(train[['TripType', 'VisitNumber', 'Weekday']].drop_duplicates(), on='VisitNumber', how = 'left')



## Another dataframe to OHE departments
new_df = train.groupby(['VisitNumber', 'DepartmentDescription'])['ScanCount'].sum().reset_index()

## OHE
temp = pd.get_dummies(cat, columns=['DepartmentDescription'])

## THis bit of code I iterated throught the department desciption columns and made an
## aggregate function to make only one row per trip with each deparment description it 
## had showing as one
new_dict = {}
for i in temp:
    new_dict[i] = 'sum'
final = temp.groupby(temp['VisitNumber']).aggregate(new_dict)
final = final.iloc[: , 1:].reset_index()



## Merging the final df and merged df
temp = merged.merge(final, on='VisitNumber')


NameError: ignored

## Final Test Set 
We will reserve 10% of our data for final validation. This will serve as our overall task performance benchmark.

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(train, test_size = 0.1)

##Partitioning Train/Test Sets:
Initially, we decided that our train/test data splitting will be implemented using Kfold CV. But since we are working very large dataset (500k+ datapoints), we found we had insufficent compute resource to run too many iterations. Now, the data splitting will be done through a simple train-test partition with 70% of the datapoints in the training set and 30% in the validation set. 

In [ ]:
from sklearn.model_selection import train_test_split
X = train.dropna()
#X = temp
y = X['TripType']
X = X.drop(columns=['TripType'])
X = X.drop(columns=["DepartmentDescription"])
#y = temp['TripType']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .70, shuffle=True)
print (y_test.unique().shape)
print(X_train.shape)


(38,)
(450047, 5)


# Proposed Solution

Our proposed solution is to use train and fit a model to the training data, evaluate its performance on the testing data, and then use cross validation to verify the accuracy of our results.

Given the nature of our problem, non-binary classification of trip type into 38 distinct categories, we have isolated some models we believe would be best suited for the task:

- SVM
- Random Forest 
- Neural Networks

####Note On Model Selection:
The nature of our dataset naturally lends itself to certain models and leave other ones unviable. Because of the number of features we have and the large dataset, K-NN Classifiers would suffer. Logistic regression is another such classifer that we can rule out due to the non binary catagorization of our dataset. 

we also are constrained in our compute time and resource, this means that certain classifiers that have more complex training times must be evaluated more harshly.


# Evaluation Metrics

Given the context of our data and intended solution, we need a multiclass evaluation metric. Thus, we've chosen to evaluate the performance of our benchmark model and solution models by calculating the F1-score. The F1-score combines two metrics, precision and recall, using their harmonic mean to give one single number. Therefore, the F1-score would allow us to take both the number of false positives and false negatives of trip type classification into account. Further, the F1-score is a useful evaluation metric to compare against the accuracy scores of our proposed models, considering that we have an uneven class distribution. For this reason, we chose to calculate the F1-score with a weighted average to take into account the proportion of each trip type label in our dataset.



#Model Training
In search of the optimal model, we will instantiate each of our hypothesized algorithms and perform a hyperparameter sweep on each to select the best parameters. This allows each classifier to put it's best foot forward during their eventual model comparison.




#####*Note on Constrained Model Training:
Given the compute resource contraint, we will be training all of our models with a class balanced random sample (75k samples) of the larger training dataset. This is to allow for faster training times, allowing more granular hyperparameter tuning. We will then train/test the best model on the entire dataset.

## Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#using random forest to classify data with dimensionality reduced to first feature
rfc = RandomForestClassifier(max_depth=20)
rfc.fit(X_train, y_train)

# Predicting the Test set results
pred = rfc.predict(X_test)
#pred

In [ ]:
rfc.score(X_test, y_test)

0.3425948008585738


#Hi 

In [ ]:
from sklearn. metrics import classification_report

print(classification_report(y_test, pred))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("Overall Accuracy:",accuracy_score(y_test, pred))
print("Overall Precision:",precision_score(y_test, pred, average='macro'))
print("Overall Recall:",recall_score(y_test, pred, average='macro'))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test, pred)

print(confusion_matrix)

###Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

reg_params = {
"n_estimators": [100,200,50]

},

GSCV = GridSearchCV(rfc,reg_params,scoring="f1",verbose =3)
GSCV.fit(X_train,y_train)

## Neural Networks using SKLearn

###Base Case

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
NN = MLPClassifier(hidden_layer_sizes=(150,100), solver='adam', random_state=5671, activation ="relu", learning_rate="adaptive")
kf=KFold(5)
NN.fit(X_train,y_train)
#cross_validate(NN, X_train,y_train, scoring=("f1","accuracy"), cv=kf)
pred = NN.predict(X_test)
pred 

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


array([40, 40, 40, ..., 40, 40, 40])

In [ ]:
from sklearn. metrics import classification_report

print(classification_report(y_test, pred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           3       0.00      0.00      0.00      2003
           4       0.00      0.00      0.00       261
           5       0.00      0.00      0.00      3367
           6       0.00      0.00      0.00      1048
           7       0.00      0.00      0.00      6956
           8       0.00      0.00      0.00      6872
           9       0.00      0.00      0.00      5051
          12       0.00      0.00      0.00       660
          14       0.00      0.00      0.00        13
          15       0.00      0.00      0.00      2133
          18       0.00      0.00      0.00       896
          19       0.00      0.00      0.00       355
          20       0.00      0.00      0.00       923
          21       0.00      0.00      0.00      1229
          22       0.00      0.00      0.00      1095
          23       0.00      0.00      0.00        93
          24       0.00      0.00      0.00      5368
          25       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("Overall Accuracy:",accuracy_score(y_test, pred))
print("Overall Precision:",precision_score(y_test, pred, average='macro'))
print("Overall Recall:",recall_score(y_test, pred, average='macro'))

Overall Accuracy: 0.2704092742562656
Overall Precision: 0.007116033533059621
Overall Recall: 0.02631578947368421


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, pred)
print(confusion_matrix)

###Hyperparameter Tuning 

In [ ]:
from sklearn.model_selection import GridSearchCV

NN_params = {
    
}

#Model Comparison

## Comparing models with cross validation

In [ ]:
# Import required libraries for performance metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate

# Import required libraries for machine learning classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


# Define dictionary with performance metrics we want to use
score = {'accuracy':make_scorer(accuracy_score), 
           'precision':make_scorer(precision_score),
           'recall':make_scorer(recall_score), 
           'f1_score':make_scorer(f1_score)}

# Initializing the machine learning classifiers
log_model = LogisticRegression(C=1/0.1, solver='lbfgs', multi_class='auto', max_iter=10000)
rfc_model = RandomForestClassifier()
nn_model = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

def model_evaluation(X, y, kfolds):
    
    
    # Performing cross-validation to each machine learning classifier
    
    rfc = cross_validate(rfc_model, X, y, cv=kfolds, scoring=score)
    nn = cross_validate(nn_model, X, y, cv=kfolds, scoring=score)

    # Create a data frame with the models performance metrics scores
    models_scores_frame = pd.DataFrame({'Logistic Regression':[log['test_accuracy'].mean(),
                                                               log['test_precision'].mean(),
                                                               log['test_recall'].mean(),
                                                               log['test_f1_score'].mean()],
                                       
                                      'Random Forests':[rfc['test_accuracy'].mean(),
                                                       rfc['test_precision'].mean(),
                                                       rfc['test_recall'].mean(),
                                                       rfc['test_f1_score'].mean()],
                                       
                                      'Neural Networks':[nn['test_accuracy'].mean(),
                                                              nn['test_precision'].mean(),
                                                              nn['test_recall'].mean(),
                                                              nn['test_f1_score'].mean()]},
                                      
                                      index=['Accuracy', 'Precision', 'Recall', 'F1 Score'])
    
      # Add 'Best Score' column
    models_scores_frame['Highest Score'] = models_scores_frame.idxmax(axis=1)
    
    # Return models performance metrics scores data frame
    return(models_scores_frame)
  
# Run models_evaluation function
model_evaluation(X, y, 5)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(50, input_dim=5, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

NN= KerasClassifier(build_fn=baseline_model, epochs =200,batch_size=5,verbose=2)

In [ ]:
results = cross_val_score(, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

# Ethics & Privacy

Our data does not involve anyones data or identity so it would be difficult to find a breach of ethics or privacy. One ethical dilemma that might arise is the increasing amount of data available to big corporations and how they’re using this big data to fine tune their products to keep the average person consuming even more. It would be good to question whether it is completely ethical for corporations to treat everyone as a number to maximize their profits.

Another note on privacy is the trip type classification labels. 



# Team Expectations 

Put things here that cement how you will interact/communicate as a team, how you will handle conflict and difficulty, how you will handle making decisions and setting goals/schedule, how much work you expect from each other, how you will handle deadlines, etc...

- Arrange bi-weekly meetings that works with everyones schedule
- Use a discord server to communicate with one another
- Make use of project managment software to track progress
- Be mindful of git -pull-push-overwrites such that no code is overwritten or needlessly repeated

# Project Timeline Proposal

UPDATE THE PROPOSAL TIMELINE ACCORDING TO WHAT HAS ACTUALLY HAPPENED AND HOW IT HAS EFFECTED YOUR FUTURE PLANS

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 4/24  |  3:30 PM |  Edit, finalize, and submit proposal; Search for datasets  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part | 
| 5/16  |  9 PM |  Delegate tasks for first checkpoint and discuss wrangling, cleaning, and EDA plan | Import and wrangle data, do some EDA | 
| 5/19  | 9 PM  | Edit and finalize data cleaning and wrangling/EDA  | Review/discuss EDA, debug, and submit checkpoint   |
| 5/23  | 7 PM  | Finalize project/conclusion/discussion | Discuss conclusion   |
| 6/8  | Before 11:59 PM  | NA | Turn in Final Project  |

# Footnotes
<a name="first"></a>1.[^](#firstnote): Oliver Kramer. K Nearest Neighbors. https://link.springer.com/chapter/10.1007/978-3-642-38652-7_2<br> 
<a name="second"></a>2.[^](#secondnote): Srivastava et al. (1999) Parallel Forumlations of Decision Tree Classfication Algorithms. http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.41.7475&rep=rep1&type=pdf<br>
<a name="third"></a>3.[^](#thirdnote): M. Pal (2005).Random forest classifier for remote sensing classification. https://www.tandfonline.com/doi/pdf/10.1080/01431160412331269698?casa_token=e78vG4sBDLcAAAAA:p9nt0mSjEMuazyQsDjprmwIIFt9aNRk9EtF7eKRyNozF6FsAskuvXKrMxnnftOK0xFjlUm5MX9g.<br>
<a name="fourth"></a>4.[^](#fourthnote): Stephan Dreiseitl and Lucila Ohno_Machado. (2002). Logistic regression and artificial neural network classification models: a methodology review. https://www.sciencedirect.com/science/article/pii/S1532046403000340.<br>
<a name="fifth"></a>5.[^](#fifthnote): Cui et al. (2018). Deep Embedding Logistic Regression. https://ieeexplore.ieee.org/document/8588790

